In [ ]:
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
import pandas as pd
import time
import json
from bs4 import BeautifulSoup

In [96]:
import flask
import flask_cors
import pandas as pd
import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
def to_json(data):
    json_data = {}
    first = True
    for row in data:
        if first: 
            first = not first
            continue
        if "+" in row[2]:continue
        json_data[row[2].replace("\u200b","")] = {
            "profit":{
                "percent":row[1].split("  ")[0],
                "interval":row[1].split("  ")[1]},
            "datetime":row[3],
            "event":row[4],
            "market":row[5],
            "odds":row[6],
        }
    return json_data

def scrape1xbet():
    df=pd.DataFrame(columns=['teams','team1_odds','team2_odds','site'])
    browser= webdriver.Edge()
    browser.get('https://parimatch-in.com/en/cricket/live')
    time.sleep(15)
    data = browser.find_elements(By.CLASS_NAME, "EC_AI")
    for i in data:
        soup = BeautifulSoup(i.get_attribute('innerHTML'), 'html.parser')
        match=soup.find_all(class_='styles_wrapper__BfdYz styles_wrapper-card-with-favorites__kCilz styles_wrapper-card__2uN8L EC_Hb')
        for i in match:
            name=i.find_all(class_='styles_link__1wxWs')
            a=i.find_all(class_='styles_value__1V_3B styles_value__3bNG1') 
            df=df.append({'teams':name[0].text,'team1_odds':a[0].text,'team2_odds':a[1].text,'site':'parimatch'},ignore_index=True)
    browser.close()
    return df
def scrapeparimathc():
    df=pd.DataFrame(columns=['teams','team1_odds','team2_odds','site'])
    browser= webdriver.Edge()
    browser.get('https://parimatch-in.com/en/cricket/live')
    time.sleep(15)
    data = browser.find_elements(By.CLASS_NAME, "EC_AI")
    for i in data:
        soup = BeautifulSoup(i.get_attribute('innerHTML'), 'html.parser')
        match=soup.find_all(class_='styles_wrapper__BfdYz styles_wrapper-card-with-favorites__kCilz styles_wrapper-card__2uN8L EC_Hb')
        for j in match:
            name=j.find_all(class_='styles_link__1wxWs')
            a=i.find_all(class_='styles_value__1V_3B styles_value__3bNG1')
            df=df.append({'teams':name[0].text+name[1].text,'team1_odds':a[0].text,'team2_odds':a[1].text,'site':'parimatch'},ignore_index=True)
    browser.close()
    return df
def scrapemagapari():
    df=pd.DataFrame(columns=["teams","team 1 odds","team 2 odds","website"])
    browser = webdriver.Edge()
    browser.get('https://megapari.com/live/cricket')
    time.sleep(10)
    data=browser.find_element(By.CSS_SELECTOR, '#games_content')
    soup = BeautifulSoup(data.get_attribute('innerHTML'), 'html.parser')
    matches = soup.findAll("div",class_='dashboard-champ-content')
    for match in matches:
        a=match.findAll("span",class_="c-bets__inner")
        name=match.findAll("div",class_="c-events__team")
        df=df.append({"teams":name[0].text,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"megapari"},ignore_index=True)
    browser.quit()
    return df

def scrapebetway():
    df=pd.DataFrame(columns=["teams","team 1 odds","team 2 odds","website"])
    browser = webdriver.Edge()
    browser.get('https://1x-bet.in/en/live/cricket')
    time.sleep(5)
    data=browser.find_element(By.CSS_SELECTOR, '.ui-dashboard')
    soup = BeautifulSoup(data.get_attribute('innerHTML'), 'html.parser')
    matches = soup.findAll(class_='ui-dashboard-champ dashboard-champ dashboard__champ ui-dashboard-champ--theme-gray')
    for match in matches:
        a=match.findAll(class_="market__value")
        name=match.findAll(class_="dashboard-game-team-info dashboard-game-block__team")
        df=df.append({"teams":name[0].text+" vs "+name[1].text,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"betway"},ignore_index=True)
    browser.quit()
    return df
def scrapebet365():
    df=pd.DataFrame(columns=["teams","team 1 odds","team 2 odds","website"])
    browser = webdriver.Edge()
    browser.get('https://1x-bet.in/en/live/cricket')
    time.sleep(5)
    data=browser.find_element(By.CSS_SELECTOR, '.ui-dashboard')
    soup = BeautifulSoup(data.get_attribute('innerHTML'), 'html.parser')
    matches = soup.findAll(class_='ui-dashboard-champ dashboard-champ dashboard__champ ui-dashboard-champ--theme-gray')
    for match in matches:
        a=match.findAll(class_="market__value")
        name=match.findAll(class_="dashboard-game-team-info dashboard-game-block__team")
        df=df.append({"teams":name[0].text+" vs "+name[1].text,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"bet365"},ignore_index=True)
    browser.quit()
def scrapebetfair():
    df=pd.DataFrame(columns=["teams","team 1 odds","team 2 odds","website"])
    browser = webdriver.Edge()
    browser.get('https://1x-bet.in/en/live/cricket')
    time.sleep(5)
    data=browser.find_element(By.CSS_SELECTOR, '.ui-dashboard')
    soup = BeautifulSoup(data.get_attribute('innerHTML'), 'html.parser')
    matches = soup.findAll(class_='ui-dashboard-champ dashboard-champ dashboard__champ ui-dashboard-champ--theme-gray')
    for match in matches:
        a=match.findAll(class_="market__value")
        name=match.findAll(class_="dashboard-game-team-info dashboard-game-block__team")
        df=df.append({"teams":name[0].text+" vs "+name[1].text,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"betfair"},ignore_index=True)
    browser.quit()
    return df

app = flask.Flask(__name__)
flask_cors.CORS(app)

@app.route('/api/v1/surebets', methods=['post'])
def scrapeall():
    df=scrape1xbet()
    df=df.append(scrapeparimathc(),ignore_index=True)
    df=df.append(scrapemagapari(),ignore_index=True)
    df=df.append(scrapebetway(),ignore_index=True)
    df=df.append(scrapebet365(),ignore_index=True)
    df=df.append(scrapebetfair(),ignore_index=True)
    return df.to_json(orient='records')
if __name__ == '__main__':
    app.run(debug=True)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
/home/adi/anaconda3/envs/webscraping/lib/python3.11/site-packages/traitlets/traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/home/adi/anaconda3/envs/webscraping/lib/python3.11/site-packages/traitlets/traitlets.py:2499: FutureWarning: Supporting extra quotes around Bytes is deprecated in traitlets 5.0. Use 'b03bfb12-cbf9-4235-a5c5-f34295b0319b' instead of 'b"b03bfb12-cbf9-4235-a5c5-f34295b0319b"'.
  warn(
Traceback (most recent call last):
  File "<frozen runpy>

SystemExit: 1

/home/adi/anaconda3/envs/webscraping/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [98]:
    df=pd.DataFrame(columns=["teams","team 1 odds","team 2 odds","website"])
    browser = webdriver.Edge()
    browser.get('https://megapari.com/live/cricket')
    time.sleep(10)
    data=browser.find_element(By.CSS_SELECTOR, '#games_content')
    soup = BeautifulSoup(data.get_attribute('innerHTML'), 'html.parser')
    matches = soup.findAll("div",class_='dashboard-champ-content')
    for match in matches:
        a=match.findAll("span",class_="c-bets__inner")
        name=match.findAll("div",class_="c-events__team")
        df=df.append({"teams":name[0].text,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"megapari"},ignore_index=True)
    browser.quit()

/tmp/ipykernel_4979/1521663892.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({"teams":name[0].text,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"megapari"},ignore_index=True)
/tmp/ipykernel_4979/1521663892.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({"teams":name[0].text,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"megapari"},ignore_index=True)
/tmp/ipykernel_4979/1521663892.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({"teams":name[0].text,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"megapari"},ignore_index=True)
/tmp/ipykernel_4979/1521663892.py:11: FutureWarning: The frame.append method is deprecated and will be remove

SyntaxError: 'return' outside function (1521663892.py, line 13)

In [99]:
df

,teams,team 1 odds,team 2 odds,website
0,\n York...,1.45,-,megapari
1,\n Esse...,2.75,-,megapari
2,\n Sinh...,-,-,megapari
3,\n Aven...,-,-,megapari
4,\n Mala...,1.22,-,megapari
5,\n Sach...,2.18,-,megapari
6,\n Rais...,2.165,-,megapari
7,\n Jano...,2.264,-,megapari
8,\n VVV ...,2.95,-,megapari
9,\n Kara...,1.89,25,megapari


In [92]:
df=df.append(scrapeparimatch())

/tmp/ipykernel_4979/3035332558.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({"teams":name,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"parimatch"},ignore_index=True)
/tmp/ipykernel_4979/3035332558.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({"teams":name,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"parimatch"},ignore_index=True)
/tmp/ipykernel_4979/3035332558.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({"teams":name,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"parimatch"},ignore_index=True)
/tmp/ipykernel_4979/3035332558.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

In [93]:
df

,teams,team 1 odds,team 2 odds,website
0,\n Lancashire Lightning\n ...,\n 1.44\n,\n 25\n,1xbet
1,\n Ajman D10\n vs ...,\n 7.85\n,\n 25\n,1xbet
2,\n NCM Investment\n ...,\n -\n,\n -\n,1xbet
3,\n Sari Sari Sunrisers\n ...,\n -\n,\n -\n,1xbet
4,\n Premier League U19\n ...,\n -\n,\n -\n,1xbet
5,\n JT Nasir Sports\n ...,\n 1.445\n,\n 25\n,1xbet
6,\n Masaqa Cricket Club\n ...,\n -\n,\n -\n,1xbet
7,\n Millionaires\n ...,\n 1.144\n,\n 25\n,1xbet
8,\n Southern Pirates\n ...,\n 3.05\n,\n 25\n,1xbet
9,\n Red Kings\n vs ...,\n 1.79\n,\n 25\n,1xbet


In [84]:
print(scrapeall())

/tmp/ipykernel_4979/3653080616.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({"teams":name[0].text+" vs "+name[1].text,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"1xbet"},ignore_index=True)
/tmp/ipykernel_4979/3653080616.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({"teams":name[0].text+" vs "+name[1].text,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"1xbet"},ignore_index=True)
/tmp/ipykernel_4979/3653080616.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({"teams":name[0].text+" vs "+name[1].text,"team 1 odds":a[0].text,"team 2 odds":a[1].text,"website":"1xbet"},ignore_index=True)
/tmp/ipykernel_4979/3653080616.py:39: FutureWarning: The f

AttributeError: 'NoneType' object has no attribute 'append'